In [1]:
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, BatchNormalization, merge, Conv2D
import numpy as np
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, BatchNormalization, merge, Conv2D
import keras.backend as K
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers.merge import concatenate
from keras.layers import add
from keras.layers import Input,GlobalAveragePooling2D
import keras
from keras.models import  Model
from keras.preprocessing.image import  ImageDataGenerator
import keras.callbacks as kcallbacks
import math
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
def conv_block_with_maxpool(input_img):
    tower_1 = Conv2D(32, (1, 1), padding='same', activation='relu')(input_img)
    tower_1=MaxPooling2D()(tower_1)
    tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)
    tower_1=MaxPooling2D()(tower_1)
    tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
    tower_2=MaxPooling2D()(tower_2)
    tower_2 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_2)
    tower_2=MaxPooling2D()(tower_2)
    tower_3 = Conv2D(128, (1, 1), padding='same', activation='relu')(input_img)
    tower_3=MaxPooling2D()(tower_3)
    tower_3 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_3)
    tower_3=MaxPooling2D()(tower_3)
    output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=-1)
    return output

In [3]:
def conv_block(input_img):
    tower_1 = Conv2D(32, (1, 1), padding='same', activation='relu')(input_img)
    tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)
    tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
    tower_2 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_2)
    tower_3 = Conv2D(128, (1, 1), padding='same', activation='relu')(input_img)
    tower_3 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_3)
    output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=-1)
    return output

In [5]:
def resnet(input_img):
    shortcut=input_img
    x=conv_block(input_img)
    x=add([shortcut,x])
    x=Activation('relu')(x)
    return x

In [7]:
inputs = Input(shape=(512, 512, 3))
x=Conv2D(32,(3,3),padding='valid',activation='relu')(inputs)
x=MaxPooling2D()(x)
x=conv_block_with_maxpool(x)
x=resnet(x)
x=resnet(x)
x = GlobalAveragePooling2D()(x)
x=Dense(128,activation='relu')(x)
x=Dropout(0.5)(x)
x=Dense(64,activation='relu')(x)
x=Dropout(0.5)So the more even the test set is, the more objective the results are.(x)
predictions=Dense(5,activation='softmax')(x)

In [8]:
model =Model(inputs=inputs,outputs=predictions)
plot_model(model=model,to_file='./model.jpg')

In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 510, 510, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_8 (MaxPooling2D)  (None, 255, 255, 32) 0           conv2d_20[0][0]                  
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 255, 255, 32) 1056        max_pooling2d_8[0][0]            
__________________________________________________________________________________________________
conv2d_23 

In [10]:
TARGET_SIZE=(512,512)
INPUT_SIZE=(512,512,3)
BATCHSIZE=16
##################################
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './train/',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        './test/',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')

Found 49658 images belonging to 5 classes.
Found 10000 images belonging to 5 classes.


In [11]:
savepath='./mymodel.h5'
epochs=20
#定义一下优化的方法
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#compile一下模型
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
#train model
earlyStopping=kcallbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1, mode='auto')
saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
#     model_vgg19_clair.fit(X_train_vgg,y_train,batch_size=batchsize,epochs=epochs,verbose=1,
#               validation_data=(X_test_vgg,y_test),shuffle=True,callbacks=[earlyStopping,saveBestModel])
hist=model.fit_generator(
    train_generator,
    steps_per_epoch=math.ceil(49658/BATCHSIZE),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=math.ceil(10000/BATCHSIZE),
    callbacks=[earlyStopping,saveBestModel],
    workers=8
)
with open('mymodel_log.txt','w') as f:
    f.write(str(hist.history))

Epoch 1/20
3104/3104 [==============================] - 1391s 448ms/step - loss: 1.3051 - acc: 0.4719 - val_loss: 1.1194 - val_acc: 0.5367
Epoch 2/20
3104/3104 [==============================] - 1392s 448ms/step - loss: 1.1155 - acc: 0.5411 - val_loss: 1.0928 - val_acc: 0.5427
Epoch 3/20
3104/3104 [==============================] - 1391s 448ms/step - loss: 1.0943 - acc: 0.5451 - val_loss: 1.0979 - val_acc: 0.5464
Epoch 4/20
3104/3104 [==============================] - 1391s 448ms/step - loss: 1.0905 - acc: 0.5463 - val_loss: 1.0821 - val_acc: 0.5457
Epoch 5/20
3104/3104 [==============================] - 1391s 448ms/step - loss: 1.0859 - acc: 0.5467 - val_loss: 1.0803 - val_acc: 0.5508
Epoch 6/20
3104/3104 [==============================] - 1391s 448ms/step - loss: 1.0868 - acc: 0.5494 - val_loss: 1.0819 - val_acc: 0.5468
Epoch 7/20
3104/3104 [==============================] - 1391s 448ms/step - loss: 1.0810 - acc: 0.5498 - val_loss: 1.0981 - val_acc: 0.5411
Epoch 8/20
3104/3104 [=====